# Analyzing borrowers’ risk of defaulting

# Contents <a id='back'></a>

* [Introduction](#intro)
* [Stage 1. Data overview](#data_review)    
* [Stage 2. Data preprocessing](#data_preprocessing)
    * [2.1 Missing values](#missing_values)
    * [2.2 Data transformation](#data_transformation)
    * [2.3 Duplicates](#duplicates)
    * [2.4 Data categorization](#data_categorization)
* [Stage 3. Answering the questions](#questions)
* [General Conclusion](#conclusion)

# Introduction <a id='intro'></a>
In this project, we will analyze stored by the bank data on customers’ creditworthiness. We will identify if a customer’s marital status,  income level, loan purpose, and a number of children have an impact on whether they will default on a loan. The project findings later can be used for building a credit score for potential customers to evaluate their ability to repay their loans.

### Goal: 
The goal of the the projet is to answer the following questions:
1. Is there a connection between having kids and repaying a loan on time?
2. Is there a connection between marital status and repaying a loan on time?
3. Is there a connection between income level and repaying a loan on time?
4. How do different loan purposes affect on-time loan repayment?

### Stages 
We do not have information about the quality of the data, therefore, we would need to examine and preprocess the data before answering the above-mentioned questions. 

Firstly, we will evaluate the quality of the data and identify if any significant issues need to be addressed. Secondly, we will preprocess the data: fill in/remove missing values, delete duplicate data and categorize the data. And finally, we will answer the questions raised in this project.
 
The project will consist of three stages:
 1. Data overview
 2. Data preprocessing
 3. Answering the questions
 
[Back to Contents](#back)

# Stage 1. Data overview <a id='data_review'></a>

Firstly, we will open the data and explore it.

According to the documentation the table has 12 columns
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan

Loading data:

In [1]:
# Loading libraries
import pandas as pd

# Loading the data
data=pd.read_csv('/datasets/credit_scoring_eng.csv')

Checking how many rows and columns the dataset has:

In [2]:
# obtaining data shape
data_shape_original=data.shape
data_shape_original

(21525, 12)

Printing the first 10 table rows:

In [3]:
# obtaining first 10 rows
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Printing general information about the table:

In [4]:
# getting info on data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Conclusions**

As can be seen, there are 21525 rows and 12 columns in the table. Each row contains information about individual customers including the number of children they have, work experience in days, age, education, family status, gender, employment status, income level, and purpose of the loan. Also, there is information about if the customer had any debt on loan repayment in the past. 

It can be seen that the `days_employed` column has two issues: some values there are negative, the positive value looks unrealistic as it is impossible to work 340266 days (929 years). The education column has duplicate education levels written in a different register. As well, the purpose column has similar purposes written in different words. In addition, `days_employed` and `total_income` columns have missing values. 

This issue should be addressed before we start answering the raised-above questions.

[Back to Contents](#back)

# Stage 2. Data preprocessing <a id='data_preprocessing'></a>

Before answering the raised-above questions we will preprocess the data: address missing values, transform data, remove duplicates and categorize the data.

## Missing values <a id='missing_values'></a>

First of all, we will check how many missing values the table has.

In [5]:
#getting the number of missing values in each column
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

As can be seen, `days_employed` and `total_income` columns have equal numbers of missing values. Let's display rows there data is missed in `days_employed` column:

In [6]:
# displaying rows with missing values in the the days_employed column
data[data['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


There are 2174 rows with missing values in `days_employed` column. Also, missing values in `days_employed` and `income_type` columns look symmetric: in every case where the value in `days_employed` column is missing, the value in `income_type` column is missing as well. To confirm this assumption, we will filter the data by displaying all rows where values are missed in both `days_employed` and `income_type` columns.

In [7]:
# applying multiple conditions for filtering data and looking at the number of rows in the filtered table.
data[(data['days_employed'].isna()) & (data['total_income'].isna())]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


The filtered table also has 2174 rows, which proves our assumption that values in `days_employed` and `income_type` columns are missed symmetrically. So, probably these values were not filled intentionally. 

We will explore the data with missing values in `days_employed` and `income_type` columns to find out if the data could be missed due to the specific client characteristic and if there is any dependence missing values have on the value of other indicators. We will check if there is a correlation between missing data and `education`, `family_status`, and `income_type` characteristics. To do this we will compare the percentage of values in each of these characteristics in the rows with missing data with the percentage of values in each of these characteristics in the whole dataset.

Checking correlation between missing data and `income_type`:

In [8]:
#obtaining value counts for income_type column for rows where data is missing
income_types_with_missing_values=data[data['days_employed'].isna()]['income_type'].value_counts().to_frame()

#calculating percentage of each income type in total number of rows where data is missing
income_types_with_missing_values_percent=round((income_types_with_missing_values/income_types_with_missing_values.sum())*100,2)

#obtaining value counts for income_type column for whole dataset
income_types_values=data['income_type'].value_counts().to_frame()

#calculating percentage of each income type in total number of rows in whole dataset
income_types_values_percent=round((income_types_values/income_types_values.sum())*100,2)

#displaying percentage of each income type in total number of rows where data is missing
display(income_types_values_percent)
print()

#displaying percentage of each income type in total number of rows in whole dataset
display(income_types_with_missing_values_percent)

,income_type
employee,51.66
business,23.62
retiree,17.91
civil servant,6.78
unemployed,0.01
entrepreneur,0.01
paternity / maternity leave,0.00
student,0.00


,income_type
employee,50.83
business,23.37
retiree,19.00
civil servant,6.76
entrepreneur,0.05


Checking correlation beteween missing data and `family_status`:

In [9]:
#obtaining value counts for family status column for rows where data is missing
family_status_with_missing_values=data[data['days_employed'].isna()]['family_status'].value_counts().to_frame()

#calculating percentage of each family status in total number of rows where data is missing
family_status_with_missing_values_percent=round((family_status_with_missing_values/family_status_with_missing_values.sum())*100,2)

#obtaining value counts for family status column for whole dataset
family_status_values=data['family_status'].value_counts().to_frame()

#calculating percentage of each family status in total number of rows in whole dataset
family_status_values_percent=round((family_status_values/family_status_values.sum())*100,2)

#displaying percentage of each family status in total number of rows where data is missing
display(family_status_values_percent)
print()

#displaying percentage of each family status in total number of rows in whole dataset
display(family_status_with_missing_values_percent)

,family_status
married,57.51
civil partnership,19.41
unmarried,13.07
divorced,5.55
widow / widower,4.46


,family_status
married,56.90
civil partnership,20.33
unmarried,13.25
divorced,5.15
widow / widower,4.37


Checking correlation beteween missing data and `education`:

In [10]:
#obtaining value counts for education column for rows where data is missing
education_with_missing_values=data[data['days_employed'].isna()]['education'].value_counts().to_frame()

#calculating percentage of each education type in total number of rows where data is missing
education_with_missing_values_percent=round((education_with_missing_values/education_with_missing_values.sum())*100,2)

#obtaining value counts for education column for whole dataset
education_values=data['education'].value_counts().to_frame()

#calculating percentage of each education type in total number of rows in whole dataset
education_values_percent=round((education_values/education_values.sum())*100,2)

#displaying percentage of each education type in total number of rows where data is missing
display(education_values_percent)
print()

#displaying percentage of each education type in total number of rows in whole dataset
display(education_with_missing_values_percent)

,education
secondary education,63.88
bachelor's degree,21.92
SECONDARY EDUCATION,3.59
Secondary Education,3.30
some college,3.10
BACHELOR'S DEGREE,1.27
Bachelor's Degree,1.25
primary education,1.16
Some College,0.22
SOME COLLEGE,0.13


,education
secondary education,64.77
bachelor's degree,22.82
SECONDARY EDUCATION,3.08
Secondary Education,2.99
some college,2.53
Bachelor's Degree,1.15
BACHELOR'S DEGREE,1.06
primary education,0.87
Some College,0.32
SOME COLLEGE,0.32


There does not look to be a correlation between missing data and `education`, `family_status` and `income_type` characteristics. The percentage of values in each of these characteristics in the rows with missing data is similar to the percentage of values in each of these characteristics in the whole dataset.

Next, we will check the percentage of the missing values compared to the whole dataset to decide how to proceed with the missing values.

In [11]:
#Calculating the percentage of the missing values compared to the whole dataset
round((data['days_employed'].isnull().sum()/data.shape[0])*100,2)

10.1

The rows with missing values comprise 10.1% of the dataset. Since there is no correlation between missing values and data in other fields, and also taking into consideration the fact that the dataset is big enough, we will drop rows with missing values. This should not significantly affect the result.

Dropping rows with missing values:

In [12]:
#getting the index of rows with missing values
index_total_income_na=data.loc[data['total_income'].isna()].index

#dropping rows with missing values
data=data.drop(index_total_income_na)

#printing shape and info of the new table to ensure that missing dat was deleted
print(data.shape)
print()
data.info()

(19351, 12)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19351 entries, 0 to 21524
Data columns (total 12 columns):
children            19351 non-null int64
days_employed       19351 non-null float64
dob_years           19351 non-null int64
education           19351 non-null object
education_id        19351 non-null int64
family_status       19351 non-null object
family_status_id    19351 non-null int64
gender              19351 non-null object
income_type         19351 non-null object
debt                19351 non-null int64
total_income        19351 non-null float64
purpose             19351 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 1.9+ MB


**Conclusions**

We have not identified any pattern in missing data. Taking into consideration the percent of rows with missing values and the size of the dataset, we dropped rows with missing values.

## Data transformation <a id='data_transformation'></a>

At the next step, we will have a deeper look at each other column one by one to identify and fix the issues they have with the data.

**`education` column**

Displaying all unique values in that column first:

In [13]:
# obtaining all values in education column to check if and what spellings will need to be fixed
data['education'].value_counts().to_frame()

,education
secondary education,12342
bachelor's degree,4222
SECONDARY EDUCATION,705
Secondary Education,646
some college,613
BACHELOR'S DEGREE,251
Bachelor's Degree,243
primary education,231
Some College,40
SOME COLLEGE,22


As can be seen, there are duplicate category types written in a different register. To fix it, we will cast all `education` column values to lower case.

In [14]:
# fixing the registers
data['education']=data['education'].str.lower()


Checking all the values in the column to make sure we fixed them:

In [15]:
# obtaining all values in education again after the fix
data['education'].value_counts().to_frame()


,education
secondary education,13693
bachelor's degree,4716
some college,675
primary education,261
graduate degree,6


**`children` colunm**

Displaying all unique values in that column:

In [16]:
# chcking the distribution of values in the `children` column
data['children'].value_counts().to_frame()


,children
0,12710
1,4343
2,1851
3,294
20,67
-1,44
4,34
5,8


There are two problematic values in that column: '-1' and '20'. The number of children cannot be negative, and it is unlikely that someone can have 20 children. Since there are only two anomaly values, it is most likely a technical error. Value '-1' could be a value '1' accidentally preposed with '-' sign. For value '20' it is difficult to tell what the original is was, is it '0', '2' or everything greater than '5'. The number of children could correlate with income level or with age. We will calculate the mean income amount and age for each number of children to see if there is any pattern and if based on that pattern we can determine what is the actual value for value '20' should be.

Checking if there is a correlation between income level and number of children:

In [17]:
#culculating mean income amount for each number of children
data.groupby('children')['total_income'].mean().to_frame()

,total_income
children,
-1,24618.108614
0,26422.404866
1,27396.494050
2,27496.357898
3,29322.623993
4,27289.829647
5,27268.847250
20,26995.284209


Checking if there is correlation between age ond number of children:

In [18]:
#culculating mean age for each number of children
data.groupby('children')['dob_years'].mean().to_frame()

,dob_years
children,
-1,42.431818
0,46.214083
1,38.394428
2,35.720151
3,36.261905
4,34.911765
5,38.375000
20,41.283582


There does not seem to be a pattern in the number of children based on income level and age. Therefore, we cannot determine what the correct value for value '20' should be. Replacing this value with a mean or median value may add error when answering the question about the correlation between the number of children and the ability to repay a loan, therefore it would be better to drop rows that have this value to avoid the error. In addition, the number of rows that have 20 children in the children column is just 67 out of 19351 which is less than 0.5% of the total number of rows. So, it should not significantly impact the end result. For value '-1' we will be stuck to our assumption and replace it with '1'.

In [19]:
# replacing value '-1' in children column with '1'
data.loc[data['children']==-1,'children']=1

# obtaining index of rows that have value '20' in children column
index_20_children=data.loc[data['children']==20].index

# dropping rows that have value '20' in children column
data=data.drop(index_20_children)


Confirming that data in `children` column was fixed:

In [20]:
# displaying unique values in children column
display(data['children'].value_counts().to_frame())

#displaying new data shape
print(f'Data shape: {data.shape}')

,children
0,12710
1,4387
2,1851
3,294
4,34
5,8


Data shape: (19284, 12)


**`days_employed` colunm**

Displaying distribution of values in that column:

In [21]:
#converting days in yeaers and describing data in days_employed column 
(data['days_employed']/365).describe().to_frame()

,days_employed
count,19284.000000
mean,172.989681
std,386.055566
min,-50.380685
25%,-7.528429
50%,-3.299081
75%,-0.795086
max,1100.699727


As we mentioned previously there are several issues with data in this column: more than 75% of values are negative, max work experience is higher than 1000 years, and average work experience is 172 years. 

Negative values most likely are the result of incorrect calculation where a higher date was subtracted from a lower date. Therefore, we will make them positive to fix them. 

Regarding positive values in this column, even though 75% of values are negative, the average work experience is 172 years, which indicates that most of the positive values are unrealistic. We will check if there are some real positive values. We will assume that the real maximum possible work experience is 80 years (29200 days) even though it is too high.

In [22]:
#obtaining number of rows there positive work experience is real (less than 29200 days)
data[(data['days_employed']>0) & (data['days_employed']<29200)]['days_employed'].count()

0

As can be seen all positive values in this column are unrealistic. Before deciding how to fix it we will check if there is any correlation between these positive values and other columns. Firstly, we will check if there is a correlation with income type.

In [23]:
#obtaining income_type value counts for rows where work experience is unrealistic
display(data[data.days_employed>0]['income_type'].value_counts().to_frame())

#obtaining income_type value counts for whole dataset
display(data['income_type'].value_counts().to_frame())

,income_type
retiree,3436
unemployed,2


,income_type
employee,9974
business,4558
retiree,3436
civil servant,1311
unemployed,2
student,1
entrepreneur,1
paternity / maternity leave,1


It appears that all rows, where work experience is unrealistic, have retiree or unemployed income types. In addition, as can be seen from value counts comparison with the whole data set, all retirees and unemployed customers have unrealistic work experience. Probably because they are not employed work experience for them was filled with some incorrect data. To fix it, we will get a delta between average age and average work experience. We will use only real work experience to calculate the average. Then, for retirees, we will subtract the delta from the average retirement age (65 years for males and 63 years for females in the USA) to get average work experience for them. For unemployed we will put in average work experience.

In [24]:
#culculating average age for males
average_age_males=data[(data['days_employed']<0)&(data['gender']=='M')]['dob_years'].mean()

#culculating average age for females
average_age_females=data[(data['days_employed']<0)&(data['gender']=='F')]['dob_years'].mean()

#culculating average work experience for males in years 
average_work_experince_males=data[(data['days_employed']<0)&(data['gender']=='M')]['days_employed'].mean()/-365

#culculating average work experience for females in years 
average_work_experince_females=data[(data['days_employed']<0)&(data['gender']=='F')]['days_employed'].mean()/-365

#culculating delta for males
delta_males=average_age_males-average_work_experince_males

#culculating delta for females
delta_females=average_age_females-average_work_experince_females

#putting data to data frame for better visualisation
columns_emp=['Gender','average age','average work experience','delta']
data_emp=[['Males',average_age_males,average_work_experince_males,delta_males],
          ['Females',average_age_females,average_work_experince_females,delta_females]]
df_emp=pd.DataFrame(columns=columns_emp, data=data_emp)

#displaying the data frame
df_emp

,Gender,average age,average work experience,delta
0,Males,39.091491,5.690507,33.400984
1,Females,40.259031,6.902875,33.356156


Getting average work experience for retired males and females:

In [25]:
#calculating average work experience for retired males in days
average_work_experince_males_r=(65-delta_males)*365

#calculating average work experience for retired males
average_work_experince_females_r=(63-delta_females)*365

#displaying average work experience for retired males and femails
print(f'Average work experience for retired males:{average_work_experince_males_r}')
print(f'Average work experience for retired females:{average_work_experince_females_r}')


Average work experience for retired males:11533.640869908699
Average work experience for retired females:10820.002913505625


There are only two unemployed customers, so we will replace work experience for them individually

Displaying unemployed customers:

In [26]:
#showing unemployed customers
data[data['income_type']=='unemployed']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,337524.466835,31,secondary education,1,married,0,M,unemployed,1,9593.119,buying property for renting out
14798,0,395302.838654,45,bachelor's degree,0,civil partnership,1,F,unemployed,0,32435.602,housing renovation


Fixing the data in `days_employed` column:

In [27]:
#fixing work experience values for retired males
data.loc[(data['income_type']=='retiree')&(data['gender']=='M'),'days_employed']=average_work_experince_males_r
         
#fixing work experience values for retired females         
data.loc[(data['income_type']=='retiree')&(data['gender']=='F'),'days_employed']=average_work_experince_females_r
         
#fixing work experience for enemployed customers        
data.loc[3133,'days_employed']=average_work_experince_males         
data.loc[14798,'days_employed']=average_work_experince_females
         
#fixing negative values
data.loc[data['days_employed']<0,'days_employed']*=-1

#converting days_employed type to integer
data['days_employed']=data['days_employed'].astype('int')

In [28]:
# chaking the result to make sure it's fixed
display(data.head(5))
data['days_employed'].describe().to_frame()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,10820,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


,days_employed
count,19284.000000
mean,3885.157851
std,3899.341020
min,5.000000
25%,927.000000
50%,2194.500000
75%,5547.750000
max,18388.000000


**`dob_years` colunm**

Displaying distribution of values in that column:

In [29]:
# describing data in dob_years column
data['dob_years'].describe().to_frame()

,dob_years
count,19284.000000
mean,43.262186
std,12.580911
min,0.000000
25%,33.000000
50%,42.000000
75%,53.000000
max,75.000000


As can be seen, some customers have an age of 0. 

Counting persentage of problematic values:

In [30]:
#calculating percentage of rows where age is 0
(data[data['dob_years']==0]['dob_years'].count()/len(data.index))*100

0.4667081518357187

There are less than 0.5% of rows with problematic values. Age may have not been filled on purpose for some particular income types. We will check this:

In [31]:
#obtaining value counts for income types for rows where age is 0
data[data['dob_years']==0]['income_type'].value_counts()

employee         49
business         18
retiree          17
civil servant     6
Name: income_type, dtype: int64

As can be seen, age is 0 for different income types. It is also impossible to decide the age based on data in other columns. Therefore, we will use average age to replace zeroes. As retirees are generally older than working people, we will calculate the average for these groups separately.

In [32]:
#culculating avarage age for retirees
retirees_average_age=data[(data['dob_years']!=0) & (data['income_type']=='retiree')]['dob_years'].mean()

#culculating avarage age for other income types
others_average_age=data[(data['dob_years']!=0) & (data['income_type']!='retiree')]['dob_years'].mean()

#Displaing avarage ages
print(f'Retiree average age: {retirees_average_age}')
print(f'Others average age: {others_average_age}')

Retiree average age: 59.43053524422346
Others average age: 40.00475435816165


Fixing values in `dob_years` column:

In [33]:
#replacing zeros in dob_years column for retiree income type
data.loc[(data['dob_years']==0) & (data['income_type']=='retiree'),'dob_years']=retirees_average_age.astype('int')

#replacing zeros in dob_years column for other income types
data.loc[(data['dob_years']==0) & (data['income_type']!='retiree'),'dob_years']=others_average_age.astype('int')

Checking the result to ensure that it is fixed:

In [34]:
# describing data in dob_years column
data['dob_years'].describe().to_frame()

,dob_years
count,19284.000000
mean,43.465619
std,12.237694
min,19.000000
25%,33.000000
50%,42.000000
75%,53.000000
max,75.000000


**`family_status` column**

Displaying value counts in that column:

In [35]:
# obtaining values for the column
data['family_status'].value_counts().to_frame()

,family_status
married,11097
civil partnership,3724
unmarried,2519
divorced,1082
widow / widower,862


There are no visible issues with values in that column. There are no duplicates or strange-looking values. We will leave values in that column as it is.

**`gender` column**

Displaying value counts in that column:

In [36]:
# Obtaining values for the column
data['gender'].value_counts().to_frame()

,gender
F,12711
M,6572
XNA,1


There is 1 row where gender value is inaccurate. Since it is only one row and it is impossible to guess what 'XNA' stays for, we will drop that row.

Dropping a row with 'XNA' value in `gender` column:

In [37]:
# obtaining index of a row with 'XNA' value in gender column
index=data[data['gender']=='XNA'].index

#dropping the row
data=data.drop(index)

Checking the result to ensure that it is fixed:

In [38]:
# Obtaining values for the column
data['gender'].value_counts().to_frame()

,gender
F,12711
M,6572


**`income_type` column**

Displaying value counts in that column:

In [39]:
# Obtaining values for the column
data['income_type'].value_counts().to_frame()

,income_type
employee,9974
business,4557
retiree,3436
civil servant,1311
unemployed,2
student,1
entrepreneur,1
paternity / maternity leave,1


There are no visible issues with values in that column, but values ‘unemployed’, ‘student’, ‘entrepreneur’, and ‘paternity / maternity leave’ have counts of 1 and 2 only. When we answer the question raised at the beginning of this project, we can not rely on the result gotten based on a single value for a particular income type. Therefore, we will drop these lines to avoid unreliable conclusions.

Dropping rows with ‘unemployed’, ‘student’, ‘entrepreneur’, and ‘paternity / maternity leave’ values in `income_type` column:

In [40]:
# obtaining index of a rows with unemployed, student, entrepreneur, and paternity / maternity leave values in income_type column
index=data[data['income_type'].isin(['unemployed', 'student', 'entrepreneur', 'paternity / maternity leave'])].index

#dropping the rows
data=data.drop(index)

Checking the result to ensure that it is fixed:

In [41]:
# Obtaining values for the column
data['income_type'].value_counts().to_frame()

,income_type
employee,9974
business,4557
retiree,3436
civil servant,1311


**`total_income` column**

Displaying distribution of values in that column:

In [42]:
# describing data in dob_years column
data['total_income'].describe().to_frame()

,total_income
count,19278.000000
mean,26785.906558
std,16483.533332
min,3306.762000
25%,16486.515250
50%,23203.328000
75%,32536.303500
max,362496.645000


There are no visible issues with values in that column. There are no negative values, and the highest income looks real. The only thing we will do with values in that column is to convert them to the integer type.

In [43]:
#converting total_income values type to integer
data['total_income']=data['total_income'].astype('int')

Cheking the result to ensure that it is fixed:

In [44]:
# displaying first five rows
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car purchase
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341,purchase of the house
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820,supplementary education
4,0,10820,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding


**`purpose` column**

Displaying value counts in that column:

In [45]:
# obtaining values for the column
data['purpose'].value_counts().to_frame()

,purpose
wedding ceremony,720
to have a wedding,690
having a wedding,680
real estate transactions,614
buy commercial real estate,595
purchase of the house,593
buying property for renting out,586
housing,584
building a real estate,579
transactions with commercial real estate,577


There are no visible issues with values in that column, but there are too many purposes. Therefore, we would need to combine them in a smaller number of categories later when we will categorize the data.

**Conclusions**

We had a deeper look at each other column one-by-one and fix the issues with the data in several columns:
1. In the `education` column we transformed all values to lower case to remove duplicate education types.
2. For the `children` column we deleted rows that have a value ‘20’ in that column. We also replaced ‘-1’ values with ‘1’.
3. For the `days_employed` column we replaced unrealistic values with average work experience, cast negative values to positive, and change column type to int.
4. In the `dob_years` column we replaced ‘0’ values with an average age.
5. For the `gender` column we dropped the row that had 'XNA' value.
6. For the `income_type` column we removed rows that had an ‘unemployed’, ‘student’, ‘entrepreneur’, and ‘paternity / maternity leave’ values because these values had counts of 1 and 2 only. 
7. Fore `total_income` column we change the column type to int.

## Duplicates <a id='duplicates'></a>

Cheking if there are any duplicates left:

In [46]:
# Checking for duplicates
data[(data.duplicated())]['children'].sum()

0

There are no duplicates in the data set. Let’s check the size of the dataset after all manipulation:

In [47]:
# Check the size of the dataset after manipulations with it
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19278 entries, 0 to 21524
Data columns (total 12 columns):
children            19278 non-null int64
days_employed       19278 non-null int64
dob_years           19278 non-null int64
education           19278 non-null object
education_id        19278 non-null int64
family_status       19278 non-null object
family_status_id    19278 non-null int64
gender              19278 non-null object
income_type         19278 non-null object
debt                19278 non-null int64
total_income        19278 non-null int64
purpose             19278 non-null object
dtypes: int64(7), object(5)
memory usage: 1.9+ MB


Calculating the percentage of changes:

In [48]:
#finding percentage of changes
round(((data_shape_original[0]-data.shape[0])/data_shape_original[0])*100,2)

10.44

**Conclusions**

The size of the original dataset was reduced by 10.44 percent. After all manipulations, the dataset has 19278 rows which should be sufficient to answer the questions.

## Data categorization <a id='data_categorization'></a>

It would be difficult to answer the question about if there is a connection between income level and repaying a loan on time because `total_income` column contains quantitative data with numerous values. Therefore, in order to answer that question, we will put data in that column into income categories. 

The `purpose` column has categorical values, but the number of purposes there is high and some of them look similar. So, we would combine purposes in a smaller number of purpose categories to answer the question about how different loan purposes affect on-time loan repayment. 

Also, in the current project, we do not study if there is a correlation between age category and on-time loan repayment, but for future studies, we will categorize data in the `dob_years` column as well.

**Categorizing data in `purpose` colunn**

Displaying value counts for `purpose` column:

In [49]:
# printing values for purpose column
data['purpose'].value_counts()

wedding ceremony                            720
to have a wedding                           690
having a wedding                            680
real estate transactions                    614
buy commercial real estate                  595
purchase of the house                       593
buying property for renting out             586
housing                                     584
building a real estate                      579
transactions with commercial real estate    577
housing transactions                        576
purchase of my own house                    574
property                                    571
purchase of the house for my family         569
building a property                         561
transactions with my real estate            558
construction of own property                553
buy real estate                             550
buy residential real estate                 545
housing renovation                          539
car                                     

As can be seen, there are many different values. We will display all unique values in that column:

In [50]:
# checking the unique values in purpose column
data['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'housing', 'transactions with my real estate', 'cars',
       'to become educated', 'getting an education',
       'second-hand car purchase', 'car', 'wedding ceremony',
       'to get a supplementary education', 'purchase of my own house',
       'real estate transactions', 'getting higher education',
       'to own a car', 'purchase of a car', 'profile education',
       'university education', 'buying property for renting out',
       'to buy a car', 'building a real estate', 'housing renovation',
       'going

Although there are many different purposes some of them look similar. So, we would combine them into a smaller number of purpose categories. The purposes can be grouped into the following main categories: ‘private real estate’, ‘commercial real estate’, ‘vehicle purchase’, ‘education’, and  ‘wedding’.

We will define the function to help to categorize the data in that column:

In [51]:
# defining a function to categorize the data based on common topics
def categorize_purpose(purpose):
    if purpose in ['purchase of the house','housing transactions','purchase of the house for my family','buy real estate',
                  'buy residential real estate','construction of own property','property','building a property','housing', 
                  'transactions with my real estate','purchase of my own house','real estate transactions',
                  'building a real estate','housing renovation']:
        return 'private real estate'
    elif purpose in ['buy commercial real estate','transactions with commercial real estate','buying property for renting out']:
        return 'commercial real estate'
    elif purpose in ['car purchase','buying a second-hand car','buying my own car','cars','second-hand car purchase', 'car',
                    'to own a car', 'purchase of a car','to buy a car']:
        return 'vehicle purchase'
    elif purpose in ['supplementary education','education','to become educated', 'getting an education', 
                     'to get a supplementary education','getting higher education','university education','profile education',
                     'going to university']:
        return 'education'
    elif purpose in ['to have a wedding','having a wedding','wedding ceremony']:
        return 'wedding'

Applying the function to `purpose` colunm to add `purpose_type` column:

In [52]:
# applying the function
data['purpose_type']=data['purpose'].apply(categorize_purpose)

Checking value counts in `purpose_type` column to ensure that the purposes were successfully categorized:

In [53]:
#printing value counts for purpose_type column
data['purpose_type'].value_counts()

private real estate       7966
vehicle purchase          3880
education                 3584
wedding                   2090
commercial real estate    1758
Name: purpose_type, dtype: int64

**Categorizing data in `dob_year`s column**

We will categorize data in that column by combining values in 10-year range groups. As the minimal age for applying for a loan is 19, we will put all ages below into one category. Also, as there are not too many applicants older than 70, we will put all ages higher than 69 into one category as well.

We will define the function to help to categorize the data in that column:

In [54]:
# defining a function to categorize the data based on age
def age_category(age):
    if age<20:
        return 'under 20'
    elif age<30:
        return '20-29'
    elif age<40:
        return '30-39'
    elif age<50:
        return '40-49'
    elif age<60:
        return '50-59'
    elif age<70:
        return '60-69'
    else:
        return '70+'

Testing the function:

In [55]:
# testing if the function works properly
print(age_category(5))
print(age_category(32))
print(age_category(81))

under 20
30-39
70+


Applying the function to `dob_years` colunm to add `age_category` column:

In [56]:
# applying the function
data['age_category']=data['dob_years'].apply(age_category)

Checking `age_category` column to ensure that the ages were successfully categorized:

In [57]:
# checking values in the new column
display(data.head(10))

#printing value counts for age_category column
data['age_category'].value_counts().to_frame()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type,age_category
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house,private real estate,40-49
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car purchase,vehicle purchase,30-39
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341,purchase of the house,private real estate,30-39
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820,supplementary education,education,30-39
4,0,10820,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding,wedding,50-59
5,0,926,27,bachelor's degree,0,civil partnership,1,M,business,0,40922,purchase of the house,private real estate,20-29
6,0,2879,43,bachelor's degree,0,married,0,F,business,0,38484,housing transactions,private real estate,40-49
7,0,152,50,secondary education,1,married,0,M,employee,0,21731,education,education,50-59
8,2,6929,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337,having a wedding,wedding,30-39
9,0,2188,41,secondary education,1,married,0,M,employee,0,23108,purchase of the house for my family,private real estate,40-49


,age_category
30-39,5088
40-49,4883
50-59,4183
20-29,2860
60-69,2091
70+,160
under 20,13


**Categorizing data in `total_income` column**

To categorize data in that column, we will check values distribution first.

In [58]:
# showing value distribution in total_income column
data['total_income'].describe()


count     19278.000000
mean      26785.409327
std       16483.535109
min        3306.000000
25%       16486.000000
50%       23202.500000
75%       32535.750000
max      362496.000000
Name: total_income, dtype: float64

As can be seen, 75% of applicants have an income of less than 32535. Therefore, we will categorize data in that column by combining values in 10000 range groups. Also, we will put all incomes higher than 40000 into one category.

We will define the function to help to categorize the data in that column:

In [59]:
# creating the function:
def income_category(income):
    if income<10000:
        return '10000-'
    elif income<20000:
        return '10000-19999'
    elif income<30000:
        return '20000-29999'
    elif income<40000:
        return '30000-39999'
    else:
        return '40000+'

Testing the function:

In [60]:
# testing if the function works properly
print(income_category(5000))
print(income_category(32000))
print(income_category(810000))

10000-
30000-39999
40000+


Applying the function to `total_income` colunm to add `income_level` column:

In [61]:
# applying the function
data['income_level']=data['total_income'].apply(income_category)

Checking `income_level` column to ensure that the ages were successfully categorized:

In [62]:
# checking values in the new column
display(data.head(10))

#printing value counts for income_level column
data['income_level'].value_counts().to_frame()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type,age_category,income_level
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house,private real estate,40-49,40000+
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car purchase,vehicle purchase,30-39,10000-19999
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341,purchase of the house,private real estate,30-39,20000-29999
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820,supplementary education,education,30-39,40000+
4,0,10820,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding,wedding,50-59,20000-29999
5,0,926,27,bachelor's degree,0,civil partnership,1,M,business,0,40922,purchase of the house,private real estate,20-29,40000+
6,0,2879,43,bachelor's degree,0,married,0,F,business,0,38484,housing transactions,private real estate,40-49,30000-39999
7,0,152,50,secondary education,1,married,0,M,employee,0,21731,education,education,50-59,20000-29999
8,2,6929,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337,having a wedding,wedding,30-39,10000-19999
9,0,2188,41,secondary education,1,married,0,M,employee,0,23108,purchase of the house for my family,private real estate,40-49,20000-29999


,income_level
10000-19999,6423
20000-29999,6040
30000-39999,3093
40000+,2800
10000-,922


**Conclusions**

We categorized data in `purpose`, `dob_years`, and `total_income` columns:
1. For the `purpose` column we combined purposes in a smaller number of purpose categories.
2. For the `dob_years` we combined values in 10-year range groups.
3. For the `total_income` column we combined values in 10000 range groups.

[Back to Contents](#back)

## Stage 3. Answering the questions <a id='questions'></a>


**Is there a correlation between having children and paying back on time?**

To answer this question we will use a pivot table and calculate the count, sum, and mean of debt for customers with a different number of children.

In [63]:
# check the children data and paying back on time
data.pivot_table(index='children', values='debt', aggfunc=['count','sum','mean'])

,count,sum,mean
,debt,debt,debt
children,,,
0,12706,952,0.074925
1,4386,408,0.093023
2,1850,176,0.095135
3,294,22,0.074830
4,34,3,0.088235
5,8,0,0.000000


**Conclusion**

It looks like there is no correlation between the number of children and paying back on time. People with 3 children pay loans as well as people who do not have children. At the same time, while the percentage of people with 4 children who pay back on time is lower than the percent of people with 3 children who pay back on time, it is higher than the percent of people with 1 and 2 children who pay back on time. Although all borrowers in the dataset who have 5 children do not have debts, there are only 8 of them. Therefore, we cannot rely on that result.

**Is there a correlation between family status and paying back on time?**

To answer this question we will use a pivot table and calculate the count, sum, and mean of debt for customers with different family statuses.

In [64]:
# checking the family status data and paying back on time
data.pivot_table(index='family_status', values='debt', aggfunc=['count','sum','mean'])

,count,sum,mean
,debt,debt,debt
family_status,,,
civil partnership,3721,336,0.090298
divorced,1082,75,0.069316
married,11095,841,0.075800
unmarried,2518,253,0.100477
widow / widower,862,56,0.064965


**Conclusion**

There seems to be a correlation between family status and paying back on time. As can be seen, single people and unmarried couples have a higher debt rate than people who are married or were married. It could be because such people are more mature and can better plan their expenses.

**Is there a correlation between income level and paying back on time?**

To answer this question we will use a pivot table and calculate the count, sum, and mean of debt for customers with different income levels.

In [65]:
# checking the income level data and paying back on time
data.pivot_table(index='income_level', values='debt', aggfunc=['count','sum','mean'])

,count,sum,mean
,debt,debt,debt
income_level,,,
10000-,922,56,0.060738
10000-19999,6423,549,0.085474
20000-29999,6040,522,0.086424
30000-39999,3093,240,0.077595
40000+,2800,194,0.069286


**Conclusion**

Although people with higher income seems to have fewer debts, surprisingly, people with income level less than 10000 have the lowest debt rate. This phenomenon should be studied further.

**How does credit purpose affect the default rate?**

To answer this question we will use a pivot table and calculate the count, sum, and mean of debt for different purpose types.

In [66]:
# checking the percentages for default rate for each credit purpose
data.pivot_table(index='purpose_type', values='debt', aggfunc=['count','sum','mean'])

,count,sum,mean
,debt,debt,debt
purpose_type,,,
commercial real estate,1758,130,0.073948
education,3584,330,0.092076
private real estate,7966,582,0.073061
vehicle purchase,3880,364,0.093814
wedding,2090,155,0.074163


**Conclusion**

As can be seen, people who take a loan to buy a vehicle or for education fail to pay off the loan on time more often in comparison with people who borrow money to buy a property or for a wedding. For education higher debt rate can be explained by the inability to get enough money while studying. Finding a reason for the higher rate of debt for vehicle loans will require additional research.

[Back to Contents](#back)

# General Conclusion <a id='conclusion'></a>

In this project we addressed the following four questions:

1. Is there a connection between having kids and repaying a loan on time?
2. Is there a connection between marital status and repaying a loan on time?
3. Is there a connection between income level and repaying a loan on time?
4. How do different loan purposes affect on-time loan repayment?

After analyzing the data, we concluded that:

1. There is no correlation between having kids and paying back on time.

2. Single people and unmarried couples have a higher debt rate than people who are married or were married.

3. People with higher income seem to have fewer debts. But surprisingly, people with income less than 10000 have the lowest debt rate and this phenomenon should be studied further.

4. People who take a loan to buy a vehicle or for education fail to pay off a loan on time more often in comparison with people who borrow money to buy a property or for a wedding. While a high debt rate can be easily explained for the education loan category, finding the reason for a higher rate of debt for vehicle loans will require additional research.


[Back to Contents](#back)